# **Burger's equation**:
# $ u_t + u u_{xx} = v u_{xx} \qquad x\in[-1,1], \quad t\in [0,1], \quad u ≡ u(x,t)$

## Boundary conditions (Dirichlet):
* $ u(-1, t) = 0, \qquad t\in [0,1]$
* $ u(1, t) = 0, \qquad t\in [0,1]$

## Initial condition:
* $ u(x, 0) = -\sin(\pi x), \qquad x\in[-1,1] $

<!-- ## *ANALYTICAL SOLUTION*
## $ u(x, y, t) = e^{-13\pi^2t}\sin(3\pi x)\sin(2\pi y)  $ -->




# Imports

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")

# Helpers

## Data

In [2]:
def generate_random_numbers(min_, max_, N, dtype):
    return min_ + (max_ - min_) * torch.rand(size=(N,), dtype=dtype)


class Data():
    def __init__(self,
                 x_min, x_max,
                 t_min, t_max,
                 Nx_domain, Nt_domain,
                 Nx_init, Nt_bound,
                 N_test,
                 device='cpu',
                 dtype=torch.float32):

        self.x_min = x_min
        self.x_max = x_max
        self.t_min = t_min
        self.t_max = t_max
        self.Nx_domain = Nx_domain
        self.Nt_domain = Nt_domain
        self.Nx_init = Nx_init
        self.Nt_bound = Nt_bound
        self.N_test = N_test
        self.device = device
        self.dtype = dtype


    # *** Create in-domain points ***
    def sample_domain(self):
        # Random Grid
        x_domain = generate_random_numbers(self.x_min, self.x_max, self.Nx_domain, self.dtype)
        t_domain = generate_random_numbers(self.t_min, self.t_max, self.Nt_domain, self.dtype)
        domain_data = torch.stack((x_domain, t_domain), dim=1)
        # domain_data = torch.stack(torch.meshgrid(x_domain, t_domain)).view(2, -1).permute(1, 0).requires_grad_(True).to(self.device)
        return torch.tensor(domain_data, dtype=self.dtype, device=self.device, requires_grad=True)

    # *** Boundary Conditions ***
    def sample_boundary(self):
        # Random boundary points
        t_bound = generate_random_numbers(self.t_min, self.t_max, self.Nt_bound, self.dtype)
        x_left = - torch.ones(1, dtype=self.dtype)
        x_right = torch.ones(1, dtype=self.dtype)

        bound_data_left = torch.stack(torch.meshgrid(x_left, t_bound)).view(2, -1).permute(1, 0)
        bound_data_right = torch.stack(torch.meshgrid(x_right, t_bound)).view(2, -1).permute(1, 0)
        bound_data = torch.cat([bound_data_left, bound_data_right]).requires_grad_(True).to(self.device)

        u_bound = torch.zeros(len(bound_data), 1, dtype=self.dtype, device=self.device)

        return bound_data, u_bound


    # *** Initial Condition ***
    def sample_initial(self):
        # Random initial points
        x_init = generate_random_numbers(self.x_min, self.x_max, self.Nx_init, self.dtype)
        t_init = torch.zeros(1, dtype=self.dtype)
        init_data = torch.stack(torch.meshgrid(x_init, t_init)).view(2, -1).permute(1, 0).requires_grad_(True).to(self.device)

        u_init = - torch.sin(math.pi * x_init).to(self.device)

        return init_data, u_init

    # *** Test set ***
    def sample_test(self):
        test_data = pd.read_csv('/content/drive/MyDrive/test_data.csv').dropna().to_numpy()
        return torch.tensor(test_data, dtype=self.dtype, device=self.device, requires_grad=True)
        # x_test = self.x_min + (self.x_max - self.x_min) * torch.rand(self.N_test)
        # t_test = self.t_min + (self.t_max - self.t_min) * torch.rand(self.N_test)
        # return torch.stack([x_test, t_test], dim=1).requires_grad_(True).to(self.device)

## Network

In [3]:
class MLPBase(nn.Module):
    def __init__(self, layers, activation=nn.Tanh(), weight_init=None, bias_init=None, device='cpu'):
        super().__init__()
        self.n_layers = len(layers) - 1
        self.layers = layers
        self.activation = activation
        self.weight_init = weight_init
        self.bias_init = bias_init

        dense_layers = [
            self.dense_layer(in_features=self.layers[i], out_features=self.layers[i + 1])
            for i in range(self.n_layers - 1)]
        dense_layers.append(nn.Linear(in_features=self.layers[-2], out_features=self.layers[-1]))

        self.mlp = nn.Sequential(*dense_layers).to(device)

    def dense_layer(self, in_features, out_features):
        dense_layer = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=out_features),
        )

        if self.weight_init is not None:
            self.weight_init(dense_layer[0].weight)

        if self.bias_init is not None:
            self.bias_init(dense_layer[0].bias)

        dense_layer.add_module("activation", self.activation)
        return dense_layer


class dMLP(MLPBase):
    def __init__(self, layers, activation=nn.Tanh(), weight_init=None, bias_init=None, device='cpu'):
        super().__init__(layers, activation, weight_init, bias_init, device)

    def forward(self, x):
        return self.mlp(x)

## PINN

In [4]:
class PINN():
    def __init__(self,
                 layers,
                 activation,
                 device):

        self.v = 0.01 / math.pi

        # Define the model
        self.model = dMLP(layers=layers,
                          activation=activation,
                          weight_init=lambda m: nn.init.xavier_normal_(m.data, nn.init.calculate_gain('tanh')),
                          bias_init=lambda m: nn.init.zeros_(m.data),
                          device=device)

        # Set the optimizers
        adam = torch.optim.Adam(self.model.parameters())
        lbfgs = torch.optim.LBFGS(self.model.parameters(),
                                  lr=1,
                                  max_iter=1_000,
                                  max_eval=None,
                                  tolerance_grad=1e-07,
                                  tolerance_change=1e-09,
                                  history_size=100,
                                  line_search_fn='strong_wolfe')

        self.optimizers = {"adam": adam, "lbfgs": lbfgs}

        # Set the Loss function
        self.criterion = nn.MSELoss()

        # Set the MAE criterion for test data only
        self.l1_loss = nn.L1Loss()


    def forward(self, x):
        return self.model(x)


    def grad(self, output, input):
        return torch.autograd.grad(
                    output, input,
                    grad_outputs=torch.ones_like(output),
                    retain_graph=True,
                    create_graph=True
                )[0]


    def calculate_pde_residual(self, x):
        # Forward pass
        u = self.forward(x)

        # Calculate 1st and 2nd derivatives
        du_dX = self.grad(u, x)
        du_dXX = self.grad(du_dX, x)

        # Retrieve the partial gradients
        du_dt = du_dX[:, 1].flatten()
        du_dx = du_dX[:, 0].flatten()
        du_dxx = du_dXX[:, 0].flatten()

        return du_dt + u.flatten() * du_dx - self.v * du_dxx


    def calculate_pde_loss(self, data):
        # Calculate the domain loss
        pde_res = self.calculate_pde_residual(data)
        pde_target = torch.zeros_like(pde_res)
        return self.criterion(pde_res, pde_target)


    def calculate_real_loss(self, real_data):
        # Calculate boundary loss
        loss_b = self.criterion(
            self.forward(real_data["bound_data"]).flatten(),
            real_data["u_bound"].flatten()
        )

        # Calculate initial loss
        loss_i = self.criterion(
            self.forward(real_data["init_data"]).flatten(),
            real_data["u_init"].flatten()
        )

        # Calculate the domain loss
        loss_pde = self.calculate_pde_loss(real_data["domain_data"])

        # Calculate total pinn loss
        return loss_b + loss_i + loss_pde


    def calculate_test_loss(self, test_data):
        pde_res = self.calculate_pde_residual(test_data)
        pde_target = torch.zeros_like(pde_res)
        return self.l1_loss(pde_res, pde_target)


    def train_on_real(self, real_data):
        loss_real = self.calculate_real_loss(real_data)
        loss_real.backward()
        return loss_real


    def closure(self):
        self.lbfgs_optimizer.zero_grad()
        return self.train_on_real(self.real_data)

## GAN-PINN

In [5]:
class GAN_PINN():
    def __init__(self,
                 x_min, x_max,
                 t_min, t_max,
                 Nx_domain, Nt_domain,
                 Nx_init, Nt_bound,
                 N_test, N_noise,
                 d_layers, d_activation,
                 checkpoint_path,
                 device='cpu',
                 dtype=torch.float32):

        # Constants
        self.checkpoint_path = checkpoint_path
        self.device = device
        self.dtype = dtype
        self.N_noise = N_noise
        self.N_test = N_test

        # Create real data
        self.real_data_init = Data(x_min, x_max,
                                   t_min, t_max,
                                   Nx_domain, Nt_domain,
                                   Nx_init, Nt_bound,
                                   N_test,
                                   device,
                                   dtype)

        # Create test data
        self.test_data = self.real_data_init.sample_test()

        # Create pinn
        self.pinn = PINN(d_layers, d_activation, device)


    def generate_data(self):
        # Create real data
        real_data = {}
        real_data["domain_data"] = self.real_data_init.sample_domain()
        real_data["bound_data"], real_data["u_bound"] = self.real_data_init.sample_boundary()
        real_data["init_data"], real_data["u_init"] = self.real_data_init.sample_initial()

        return real_data


    def train_with_adam(self, N_adam, real_data):
        optimizer = self.pinn.optimizers['adam']

        for epoch in range(1, N_adam + 1):
            optimizer.zero_grad()
            loss_D = self.pinn.train_on_real(real_data)
            optimizer.step()


    def train_with_lbfgs(self, N_lbfgs, real_data):
        self.pinn.lbfgs_optimizer = self.pinn.optimizers["lbfgs"]
        self.pinn.real_data = real_data

        for epoch in range(1, N_lbfgs + 1):
            loss_D = self.pinn.lbfgs_optimizer.step(self.pinn.closure)

        return loss_D


    def checkpoint(self):
        torch.save({
            "model": self.pinn.model.state_dict()
        }, self.checkpoint_path)


    def format_loss(self, loss):
        if loss == 0:
            return "0.0e+00"

        # Calculate the exponent part
        exponent = int(math.log10(abs(loss)))

        # Determine the format based on the value of the loss
        if abs(loss) < 1:
            formatted_loss = f"{loss:.2e}"
        else:
            # Adjust the sign of the formatted loss
            sign = "-" if loss < 0 else ""

            # Calculate the number of decimal places
            decimal_places = 2 - exponent

            # Ensure at least two decimal places
            decimal_places = max(decimal_places, 2)

            # Format the loss with the correct sign
            formatted_loss = f"{sign}{abs(loss):.{decimal_places}e}"

        return formatted_loss


    def keep_checkpoints_and_print_losses(self, iter, patience, print_every,
                                          loss_D, loss_test):

        loss_D_str = self.format_loss(loss_D)
        loss_test_str = self.format_loss(loss_test)

        if iter == 1:
            self.best_val_loss = loss_test
            self.best_epoch = -1
            self.checkpoint()
            self.flag = 1
            print(f"Iteration: {iter} | loss_D: {loss_D_str} | test_mae: {loss_test_str} - *Checkpoint*")
        else:
            if loss_test < self.best_val_loss:
                self.best_val_loss = loss_test
                self.best_epoch = iter
                self.checkpoint()
                self.flag = 1
                if iter % print_every == 0:
                    print(f"Iteration: {iter} | loss_D: {loss_D_str} | test_mae: {loss_test_str} - *Checkpoint*")
            elif iter - self.best_epoch > patience:
                if iter % print_every == 0:
                    self.early_stopping_applied = 1
                    print(f"Iteration: {iter} | loss_D: {loss_D_str} | test_mae: {loss_test_str}")
                return

        if (self.flag == 0) and (iter % print_every == 0):
            print(f"Iteration: {iter} | loss_D: {loss_D_str} | test_mae: {loss_test_str}")


    def train(self, iters, patience, print_every, N_adam, N_lbfgs):

        print(f"GAN-PINN: {iters} iterations")
        print(f"a. PINN: {N_adam} epochs --> Adam")
        print(f"b. PINN: {N_lbfgs} epochs --> L-BFGS")

        for iter in tqdm(range(1, iters + 1)):
            self.flag = 0
            self.early_stopping_applied = 0

            real_data = self.generate_data()

            self.train_with_adam(N_adam, real_data)
            loss_D = self.train_with_lbfgs(N_lbfgs, real_data)

            loss_test = self.pinn.calculate_test_loss(self.test_data)

            self.keep_checkpoints_and_print_losses(iter, patience, print_every,
                                                   loss_D, loss_test)

            if self.early_stopping_applied:
                print(f"\nEarly stopping applied at epoch {iter}.")
                break

In [7]:
# train_data = gan_pinn.generate_data()
# print(train_data.keys())

In [8]:
# key = 'init_data'
# data = train_data[key].detach().cpu().numpy()
# print(data.shape)

# plt.figure()
# plt.scatter(data[:, 0], data[:, 1])
# plt.show()

In [9]:
# key = 'u_init'
# data = train_data[key].detach().cpu().numpy().flatten()
# print(data.shape)

# plt.figure()
# plt.plot(np.sort(data))
# plt.show()

# Main

In [12]:
# Data
x_min, x_max = -1, 1
t_min, t_max = 0, 1
Nx_domain = 2_500
Nt_domain = 2_500
Nx_init = 100
Nt_bound = 100
N_noise = 1_000
N_test = 100_000

# pinn
Nd_layers = 3
Nd_neurons = 20
d_layers = [2] + Nd_layers * [Nd_neurons] + [1]
d_activation = nn.Tanh()

# Basic
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
checkpoint_path = "pinn.pth"
dtype = torch.float32
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# GAN-PINN initialization
gan_pinn = GAN_PINN(
    x_min, x_max,
    t_min, t_max,
    Nx_domain, Nt_domain,
    Nx_init, Nt_bound,
    N_test, N_noise,
    d_layers, d_activation,
    checkpoint_path
)

# Training
iterations = 30
patience = iterations
print_every = 1
num_epochs_adam = 1_000
num_epochs_lbfgs = 1

gan_pinn.train(iterations, patience, print_every, num_epochs_adam, num_epochs_lbfgs)

GAN-PINN: 30 iterations
a. PINN: 1000 epochs --> Adam
b. PINN: 1 epochs --> L-BFGS


  0%|          | 0/30 [00:00<?, ?it/s]

Iteration: 1 | loss_D: 1.05e-01 | test_mae: 2.48e-02 - *Checkpoint*
Iteration: 2 | loss_D: 2.39e-03 | test_mae: 1.56e-02 - *Checkpoint*
Iteration: 3 | loss_D: 8.19e-04 | test_mae: 1.25e-02 - *Checkpoint*
Iteration: 4 | loss_D: 2.98e-04 | test_mae: 1.26e-02
Iteration: 5 | loss_D: 5.30e-04 | test_mae: 1.18e-02 - *Checkpoint*
Iteration: 6 | loss_D: 2.68e-04 | test_mae: 9.14e-03 - *Checkpoint*
Iteration: 7 | loss_D: 7.32e-05 | test_mae: 1.31e-02
Iteration: 8 | loss_D: 3.99e-04 | test_mae: 8.77e-03 - *Checkpoint*
Iteration: 9 | loss_D: 1.59e-04 | test_mae: 1.09e-02
Iteration: 10 | loss_D: 1.93e-04 | test_mae: 6.54e-03 - *Checkpoint*
Iteration: 11 | loss_D: 7.23e-05 | test_mae: 7.11e-03
Iteration: 12 | loss_D: 2.60e-04 | test_mae: 6.64e-03
Iteration: 13 | loss_D: 6.94e-05 | test_mae: 7.47e-03
Iteration: 14 | loss_D: 1.42e-04 | test_mae: 1.39e-02
Iteration: 15 | loss_D: 2.62e-04 | test_mae: 5.53e-03 - *Checkpoint*
Iteration: 16 | loss_D: 6.05e-05 | test_mae: 5.27e-03 - *Checkpoint*
Iteration:

KeyboardInterrupt: 